In [5]:
"0" + "1" * 5,bin( int("0" + "1" * 5,2))

('011111', '0b11111')

In [7]:
"1" + "0" * 5, bin( int("1" + "0" * 5,2))

('100000', '0b100000')

In [12]:
encoded_bitarray = ""
encoded_bitarray += ("0" + "1" * 6)

In [13]:
encoded_bitarray

'0111111'

In [15]:
import numpy as np
x = np.array([16 ,16, 16, 16, 16, 16, 16 ,16 ,16, 16, 16, 16, 16,
          16, 16, 16, 16,  0 , 0 , 0 , 0 , 0  ,0 , 0,
  0])
np.sum(x)

272

In [46]:
from decimal import Decimal
# from .bitarray_util import *

class ArithmeticEncoding:
    def __init__(self, frequency_table, save_stages=False):
        self.probability_table = self.get_probability_table(frequency_table)

    def get_probability_table(self, frequency_table):
        """
        Calculates the probability table out of the frequency table.

        frequency_table: A table of the term frequencies.

        Returns the probability table.
        """
        total_frequency = sum(list(frequency_table.values()))

        probability_table = {}
        for key, value in frequency_table.items():
            probability_table[key] = value/total_frequency

        return probability_table

    def get_encoded_value(self, last_stage_probs):
        """
        After encoding the entire message, this method returns the single value that represents the entire message.

        last_stage_probs: A list of the probabilities in the last stage.
        
        Returns the minimum and maximum probabilites in the last stage in addition to the value encoding the message.
        """
        last_stage_probs = list(last_stage_probs.values())
        last_stage_values = []
        for sublist in last_stage_probs:
            for element in sublist:
                last_stage_values.append(element)

        last_stage_min = min(last_stage_values)
        last_stage_max = max(last_stage_values)
        encoded_value = (last_stage_min + last_stage_max)/2

        return last_stage_min, last_stage_max, encoded_value

    def process_stage(self, probability_table, stage_min, stage_max):
        """
        Processing a stage in the encoding/decoding process.

        probability_table: The probability table.
        stage_min: The minumim probability of the current stage.
        stage_max: The maximum probability of the current stage.
        
        Returns the probabilities in the stage.
        """

        stage_probs = {}
        stage_domain = stage_max - stage_min
        for term_idx in range(len(probability_table.items())):
            term = list(probability_table.keys())[term_idx]
            term_prob = Decimal(probability_table[term])
            cum_prob = term_prob * stage_domain + stage_min
            stage_probs[term] = [stage_min, cum_prob]
            stage_min = cum_prob
        return stage_probs

    def encode(self, msg, probability_table):
        """
        Encodes a message using arithmetic encoding.

        msg: The message to be encoded.
        probability_table: The probability table.

        Returns the encoder, the floating-point value representing the encoded message, and the maximum and minimum values of the interval in which the floating-point value falls.
        """
        
        # msg = list(msg)

        encoder = []

        stage_min = Decimal(0.0)
        stage_max = Decimal(1.0)

        for msg_term_idx in range(len(msg)):
            stage_probs = self.process_stage(probability_table, stage_min, stage_max)

            msg_term = msg[msg_term_idx]
            stage_min = stage_probs[msg_term][0]
            stage_max = stage_probs[msg_term][1]

        last_stage_probs = self.process_stage(probability_table, stage_min, stage_max)

        interval_min_value, interval_max_value, encoded_msg = self.get_encoded_value(last_stage_probs)

        return encoded_msg, encoder, interval_min_value, interval_max_value

    def process_stage_binary(self, float_interval_min, float_interval_max, stage_min_bin, stage_max_bin):
        """
        Processing a stage in the encoding/decoding process.

        float_interval_min: The minimum floating-point value in the interval in which the floating-point value that encodes the message is located.
        float_interval_max: The maximum floating-point value in the interval in which the floating-point value that encodes the message is located.
        stage_min_bin: The minimum binary number in the current stage.
        stage_max_bin: The maximum binary number in the current stage.

        Returns the probabilities of the terms in this stage. There are only 2 terms.
        """

        stage_mid_bin = stage_min_bin + "1"
        stage_min_bin = stage_min_bin + "0"

        stage_probs = {}
        stage_probs[0] = [stage_min_bin, stage_mid_bin]
        stage_probs[1] = [stage_mid_bin, stage_max_bin]

        return stage_probs

    def encode_binary(self, float_interval_min, float_interval_max):
        """
        Calculates the binary code that represents the floating-point value that encodes the message.

        float_interval_min: The minimum floating-point value in the interval in which the floating-point value that encodes the message is located.
        float_interval_max: The maximum floating-point value in the interval in which the floating-point value that encodes the message is located.

        Returns the binary code representing the encoded message.
        """

        binary_encoder = []
        binary_code = None

        stage_min_bin = "0.0"
        stage_max_bin = "1.0"

        stage_probs = {}
        stage_probs[0] = [stage_min_bin, "0.1"]
        stage_probs[1] = ["0.1", stage_max_bin]
        
        while True:
            if float_interval_max < bin2float(stage_probs[0][1]):
                stage_min_bin = stage_probs[0][0]
                stage_max_bin = stage_probs[0][1]
            else:
                stage_min_bin = stage_probs[1][0]
                stage_max_bin = stage_probs[1][1]

            stage_probs = self.process_stage_binary(float_interval_min,
                                                    float_interval_max,
                                                    stage_min_bin,
                                                    stage_max_bin)

            # print(stage_probs[0][0], bin2float(stage_probs[0][0]))
            # print(stage_probs[0][1], bin2float(stage_probs[0][1]))
            if (bin2float(stage_probs[0][0]) >= float_interval_min) and (bin2float(stage_probs[0][1]) < float_interval_max):
                # The binary code is found.
                # print(stage_probs[0][0], bin2float(stage_probs[0][0]))
                # print(stage_probs[0][1], bin2float(stage_probs[0][1]))
                # print("The binary code is : ", stage_probs[0][0])
                binary_code = stage_probs[0][0]
                break
            elif (bin2float(stage_probs[1][0]) >= float_interval_min) and (bin2float(stage_probs[1][1]) < float_interval_max):
                # The binary code is found.
                # print(stage_probs[1][0], bin2float(stage_probs[1][0]))
                # print(stage_probs[1][1], bin2float(stage_probs[1][1]))
                # print("The binary code is : ", stage_probs[1][0])
                binary_code = stage_probs[1][0]
                break

        # binary_code = BitArray(binary_code)
        return binary_code, binary_encoder

    def decode(self, encoded_msg, msg_length, probability_table):
        """
        Decodes a message from a floating-point number.
        
        encoded_msg: The floating-point value that encodes the message.
        msg_length: Length of the message.
        probability_table: The probability table.

        Returns the decoded message.
        """

        decoder = []

        decoded_msg = []

        stage_min = Decimal(0.0)
        stage_max = Decimal(1.0)

        for idx in range(msg_length):
            stage_probs = self.process_stage(probability_table, stage_min, stage_max)

            for msg_term, value in stage_probs.items():
                if encoded_msg >= value[0] and encoded_msg <= value[1]:
                    break

            decoded_msg.append(msg_term)

            stage_min = stage_probs[msg_term][0]
            stage_max = stage_probs[msg_term][1]

        return decoded_msg, decoder

def float2bin(float_num, num_bits=None):
    """
    Converts a floating-point number into binary.

    float_num: The floating-point number. 
    num_bits: The number of bits expected in the result. If None, then the number of bits depends on the number.

    Returns the binary representation of the number.
    """

    float_num = str(float_num)
    if float_num.find(".") == -1:
        # No decimals in the floating-point number.
        integers = float_num
        decimals = ""
    else:
        integers, decimals = float_num.split(".")
    decimals = "0." + decimals
    decimals = Decimal(decimals)
    integers = int(integers)

    result = ""
    num_used_bits = 0
    while True:
        mul = decimals * 2
        int_part = int(mul)
        result = result + str(int_part)
        num_used_bits = num_used_bits + 1

        decimals = mul - int(mul)
        if type(num_bits) is type(None):
            if decimals == 0:
                break
        elif num_used_bits >= num_bits:
            break
    if type(num_bits) is type(None):
        pass
    elif len(result) < num_bits:
        num_remaining_bits = num_bits - len(result)
        result = result + "0"*num_remaining_bits

    integers_bin = bin(integers)[2:]
    result = str(integers_bin) + "." + str(result)
    return result

def bin2float(bin_num):
    """
    Converts a binary number to a floating-point number.

    bin_num: The binary number as a string.

    Returns the floating-point representation.
    """

    if bin_num.find(".") == -1:
        # No decimals in the binary number.
        integers = bin_num
        decimals = ""
    else:
        integers, decimals = bin_num.split(".")
    result = Decimal(0.0)

    # Working with integers.
    for idx, bit in enumerate(integers):
        if bit == "0":
            continue
        mul = 2**idx
        result = result + Decimal(mul)

    # Working with decimals.
    for idx, bit in enumerate(decimals):
        if bit == "0":
            continue
        mul = Decimal(1.0)/Decimal((2**(idx+1)))
        result = result + mul
    return result

In [47]:
original_msg = [5,5,5,2,5,5,3,3,3,5,2,5,5,5,2,5,5,5,5,5,5,5,5,5,2,5,5,5,5,5,5,5,5,5,5,5,5,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,5,5,5,5,5,5,5]
# original_msg = ''.join(str(i) for i in original_msg)
# original_msg

In [48]:
frequency_table = {5:3, 2:1, 3:1}
# original_msg = [5,5,5,2]
AE = ArithmeticEncoding(frequency_table=frequency_table)
encoded_msg, encoder , interval_min_value, interval_max_value = AE.encode(msg=original_msg, 
                                                                          probability_table=AE.probability_table)
binary_code, encoder_binary = AE.encode_binary(float_interval_min=interval_min_value,
                                               float_interval_max=interval_max_value)
decoded_msg, decoder = AE.decode(encoded_msg=encoded_msg, 
                                 msg_length=len(original_msg),
                                 probability_table=AE.probability_table)
# decoded_msg = "".join(decoded_msg)
decoded_msg == original_msg

True

In [57]:
binary_code[2:],binary_code

('00100101001000111001011100010100000001000111100111010001011010',
 '0.00100101001000111001011100010100000001000111100111010001011010')

In [60]:
from bitarray_util import *
BitArray(binary_code[2:]), bitarray_to_uint(BitArray(binary_code[2:])),len(BitArray(binary_code[2:]))


(bitarray('00100101001000111001011100010100000001000111100111010001011010'),
 669037178954609754,
 62)

In [67]:
BitArray('0')  + uint_to_bitarray(bitarray_to_uint(BitArray(binary_code[2:])))

bitarray('0100101001000111001011100010100000001000111100111010001011010')

In [70]:
bin2float("0.0"+(BitArray('0')  +uint_to_bitarray(bitarray_to_uint(BitArray(binary_code[2:])))).to01())

Decimal('0.1450743125792322203194972819')

In [50]:
encoded_msg, encoder, binary_code, encoder_binary

(Decimal('0.1450743125792322203110928390'),
 [],
 '0.00100101001000111001011100010100000001000111100111010001011010',
 [])

## original method

In [1]:
from arithmetic_encode import *
from arithmetic_decode import *
from bitarray_util import *
from prob_dist import ProbabilityDist as prob_dist

# mant_dict = {"A": 1, "B": 1, "C": 2}
mant_dict = {5:3, 2:1}
# {48598: 2, 19637: 2, 16292: 2, 63560: 2, 33486: 2, 2056: 2}#, 1709: 2, 41572: 2, 56401: 2, 62883: 2, 3191: 2, 34639: 2, 32891: 2, 25354: 2, 18396: 2, 34348: 2, 33685: 2, 261: 2, 60698: 2, 555: 2, 34608: 2, 123: 2, 25903: 2, 39538: 2, 45268: 2, 5674: 2, 8260: 2, 37926: 2, 39523: 2, 5216: 2, 6827: 2, 39095: 2, 41953: 2, 331: 2, 631: 2, 33902: 2, 38819: 2, 49809: 2, 7740: 2, 52500: 2, 43032: 2, 5062: 2, 3952: 2, 35270: 2, 35000: 2, 1592: 2, 2992: 2, 35072: 2, 35002: 2, 1792: 2, 1764: 2, 34247: 2, 34219: 2, 1253: 2, 1232: 2, 33848: 2, 33834: 2, 948: 2, 930: 2, 33610: 4, 763: 2, 750: 2, 33462: 2, 33449: 2, 636: 2, 608: 2, 33345: 2, 33341: 2, 529: 4, 33257: 2, 33256: 2, 465: 2, 461: 2, 33203: 2, 33197: 2, 409: 2, 392: 2, 33150: 2, 33147: 2, 354: 2, 357: 2, 33106: 2, 33097: 2, 320: 2, 324: 2, 33072: 2, 33064: 2, 283: 2, 286: 2, 33039: 4, 268: 2, 267: 2, 33018: 2, 33016: 2, 240: 2, 225: 2, 32995: 2, 32991: 2, 217: 2, 221: 2, 32979: 2, 32975: 2, 200: 2, 208: 2, 32960: 2, 32958: 2, 182: 2, 185: 2, 32946: 2, 32949: 2, 169: 2, 176: 2, 32932: 2, 32941: 2, 171: 2, 164: 2, 32925: 2, 32934: 2, 145: 2, 154: 2, 32919: 2, 32922: 4, 139: 4, 151: 2, 32904: 2, 132: 2, 138: 4, 32897: 2, 32908: 4, 124: 2, 143: 2, 32889: 2, 32906: 2, 120: 2, 32885: 2, 32900: 2, 121: 2, 137: 4, 32880: 2, 110: 2, 32879: 2, 32911: 2, 117: 2, 32881: 2, 32916: 2, 108: 4, 149: 2, 32876: 2, 32928: 2}
print(mant_dict,'mantissa dictionary')
freqs = Frequencies(mant_dict)
my_a_encoder = ArithmeticEncoder(AECParams(),freqs,FreqModelBase)
# mantissa = [48598, 19637, 16292, 63560, 33486, 2056, 1709, 41572, 56401, 62883, 3191, 34639, 32891, 25354, 18396, 34348, 33685, 261, 60698, 555, 34608, 123, 25903, 39538, 45268, 5674, 8260, 37926, 39523, 5216, 6827, 39095, 41953, 331, 631, 33902, 38819, 49809, 7740, 52500, 43032, 5062, 3952, 35270, 35000, 1592, 2992, 35072, 35002, 1792, 1764, 34247, 34219, 1253, 1232, 33848, 33834, 948, 930, 33610, 33610, 763, 750, 33462, 33449, 636, 608, 33345, 33341, 529, 529, 33257, 33256, 465, 461, 33203, 33197, 409, 392, 33150, 33147, 354, 357, 33106, 33097, 320, 324, 33072, 33064, 283, 286, 33039, 33039, 268, 267, 33018, 33016, 240, 225, 32995, 32991, 217, 221, 32979, 32975, 200, 208, 32960, 32958, 182, 185, 32946, 32949, 169, 176, 32932, 32941, 171, 164, 32925, 32934, 145, 154, 32919, 32922, 139, 151, 32904, 32922, 132, 138, 32897, 32908, 124, 143, 32889, 32906, 120, 138, 32885, 32900, 121, 137, 32880, 32908, 110, 137, 32879, 32911, 117, 139, 32881, 32916, 108, 149, 32876, 32928, 108]

# mantissa = ["A","B","C"]
p_dist = freqs.get_prob_dist()
rng = np.random.default_rng(0)
data = rng.choice(p_dist.alphabet, size=5, p=p_dist.prob_list)
mantissa =  data.tolist()
print(mantissa)

encoded_bitarray = my_a_encoder.encode_block(DataBlock(mantissa))
num_extra_bits = int(np.random.randint(100))
encoded_bitarray_extra = BitArray(encoded_bitarray)
encoded_bitarray_extra += get_random_bitarray(num_extra_bits)
# print(encoded_bitarr)
# enc_bitarr = uint_to_bitarray(int(encoded_bitarr,2))
print(encoded_bitarray,encoded_bitarray_extra)
my_a_decoder = ArithmeticDecoder(AECParams(),freqs,FreqModelBase)
decoded_arr, num_bits = my_a_decoder.decode_block(encoded_bitarray_extra)
print(decoded_arr)

{5: 3, 2: 1} mantissa dictionary
[5, 5, 5, 5, 2]
bitarray('000000000000000000000000000001010100') bitarray('00000000000000000000000000000101010000100111111010101011101010100100100100110111011110110011001011111010100000110010101111')
bitarray('00000000000000000000000000000101') --- bitarray('010000100111111010101011101010100100100100110111011110110011001011111010100000110010101111')


KeyboardInterrupt: 

In [11]:
import numpy as np
DBTOBITS = 6.02
def BitAllocWaterFillingArithmetic(bitBudget, maxMantBits, nBands, nLines, SMR):
    bits = np.zeros(nBands, dtype=int)
    valid = np.ones(nBands, dtype=bool)
    bitBudget0 = bitBudget

    # Initialize arithmetic coding variables
    low = 0.0
    high = 1.0
    scale = 2**maxMantBits

    while valid.any():
        i = np.arange(nBands)[valid][np.argmax((SMR - bits * DBTOBITS)[valid])]

        # Arithmetic coding: update the range based on the symbol probabilities
        symbol_range = high - low
        high = low + symbol_range * np.sum(nLines[:i+1]) / np.sum(nLines)
        low = low + symbol_range * np.sum(nLines[:i]) / np.sum(nLines)

        # Allocate bits based on the bit budget
        if (bitBudget - nLines[i]) >= 0:
            bits[i] += 1
            bitBudget -= nLines[i]

        if bits[i] >= maxMantBits:
            valid[i] = False
        else:
            valid[i] = False

    # Convert the final fraction to a binary representation (for simplicity, assuming 8 bits)
    binary_representation = bin(int(low * scale))[2:].zfill(maxMantBits)

    # take back single bit allocations
    had_one = bits == 1
    bits [ had_one ] = 0
    bitBudget += np . sum ( nLines [ had_one ])
    # try to give remainder to bands with >1 bit allocation
    valid = np . logical_and ( bits < maxMantBits , bits > 1)
    while valid . any ():
        i = np . arange ( nBands )[ valid ][0]
        if ( bitBudget - nLines [ i ]) >= 0:
            bits [ i ] += 1
            bitBudget -= nLines [ i ]
            if bits [ i ] >= maxMantBits :
                valid [ i ] = False
            else :
                valid [ i ] = False
    # try to give remainder to bands that had one
    valid = had_one
    while valid . any ():
        i = np . arange ( nBands )[ valid ][0]
        if ( bitBudget - 2 * nLines [ i ]) >= 0:
            bits [ i ] += 2
            bitBudget -= 2* nLines [ i ]
            if bits [ i ] >= maxMantBits :
                valid [ i ] = False
        else :
            valid [ i ] = False
    # try to give remainder to bands that got nothing
    valid = bits == 0
    while valid . any ():
        i = np . arange ( nBands )[ valid ][0]
        if ( bitBudget - 2 * nLines [ i ]) >= 0:
            bits [ i ] += 2
            bitBudget -= 2* nLines [ i ]
            if bits [ i ] >= maxMantBits :
                valid [ i ] = False
        else :
            valid [ i ] = False

    return bits, binary_representation

# Example usage:
bitBudget = 2556.08
maxMantBits = 16
nBands = 25
# nLines = np.array([4, 5, 2, 3, 6])
nLines = np.array([4, 5, 4, 4, 5, 5, 6, 6, 7, 8, 9, 10, 12, 14, 16, 19, 24, 30, 38, 47, 56, 76, 107, 149, 363])
SMR = np.array([-3.000585405015684, -2.604794746992411, 17.767051474815887, 12.181859490857278, 11.157433273370742, -26.314501814520433, -13.607326042404559, 9.625303160050443, -12.576476764235487, -16.770753807633177, -8.02971270843502, -0.7377990049634136, 0.28850103284408846, -0.7736106708324471, -1.8280490060216366, -5.325518248496884, -28.20883735009098, 8.141613221694357, 12.409258272424722, -46.69972108719738, -46.048058028255134, 18.114390972226524, -53.003603571792006, -51.29305020501537, -81.38242070789546])

result_bits, binary_rep = BitAllocWaterFillingArithmetic(bitBudget, maxMantBits, nBands, nLines, SMR)

print("Allocated bits:", result_bits)
print("Binary representation:", binary_rep)

Allocated bits: [16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16  0  0  0  0  0  0  0
  0]
Binary representation: 0101001001101110


In [147]:
# mant_dict = {0: 8766, 32768: 2245, 128: 1162, 1: 728}
mant_dict = {0: 8766, 32768: 2245, 1: 728}
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_dict.pkl', 'wb')
pickle.dump(mant_dict,file)
file.close()
# freqs = Frequencies(mant_dict)
# my_a_encoder = ArithmeticEncoder(AECParams(),freqs,FreqModelBase)
# my_a_encoder.encode_block(DataBlock([0,224]))

In [154]:
2**134 / 13194501387946017475304028028016288087817

1.6505414522776631

In [27]:
a = np.array([0,2,5,2,1,0])
if a.any() in mant_dict:
    for i, a_i in enumerate(a):
        if a_i in mant_dict:
            print(i, a_i)

0 0
4 1
5 0


In [145]:
import pickle
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_list_ch1.pkl', 'rb')
data = pickle.load(file)
file.close()

In [146]:
data

[[[array([0, 0]),
   array([1, 0]),
   array([    2, 32768]),
   array([3, 0]),
   array([4, 0]),
   array([5, 0]),
   array([6, 0]),
   array([7, 0]),
   array([8, 0]),
   array([9, 0]),
   array([10,  0]),
   array([11,  0]),
   array([12,  0]),
   array([13,  0]),
   array([14,  0]),
   array([15,  0]),
   array([16,  0]),
   array([17,  0]),
   array([18,  0]),
   array([19,  0]),
   array([20,  0]),
   array([21,  0]),
   array([22,  0]),
   array([23,  0]),
   array([24,  0]),
   array([25,  0]),
   array([26,  0]),
   array([27,  0]),
   array([28,  0]),
   array([29,  0]),
   array([30,  0]),
   array([31,  0]),
   array([32,  0]),
   array([33,  0]),
   array([34,  0]),
   array([35,  0]),
   array([36,  0]),
   array([37,  0]),
   array([38,  0]),
   array([39,  0]),
   array([40,  0]),
   array([41,  0]),
   array([42,  0]),
   array([43,  0]),
   array([44,  0]),
   array([45,  0]),
   array([46,  0]),
   array([47,  0]),
   array([48,  0]),
   array([49,  0]),
   array([50

In [96]:
np.array(data[1]).shape

(1, 25, 172, 2)

In [98]:
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_list_ch2.pkl', 'rb')
data2 = pickle.load(file)
file.close()

In [100]:
np.array(data2[1]).shape

(1, 25, 172, 2)

In [44]:
mant_dict = {0: 8766, 32768: 2245, 128: 1162, 1: 728, 129: 723, 2: 383, 130: 371, 6: 257, 3: 238, 131: 237, 4: 233, 10: 229, 132: 228, 5: 220, 7: 218, 8: 207, 133: 200, 134: 199, 137: 199, 9: 197, 136: 196, 14: 192, 18: 189, 15: 189, 11: 188, 12: 187, 20: 185, 139: 185, 140: 182, 138: 182, 19: 181, 13: 177, 17: 177, 143: 176, 28: 175, 135: 173, 16: 173, 27: 170, 141: 168, 24: 164, 144: 162, 151: 160, 142: 160, 23: 158, 145: 157, 21: 156, 147: 155, 22: 154, 152: 153, 166: 152, 26: 150, 149: 150, 31: 148, 25: 146, 33: 145, 148: 142, 159: 141, 150: 141, 146: 138, 35: 138, 43: 138, 154: 138, 29: 136, 165: 136, 155: 135, 30: 133, 34: 133, 157: 132, 156: 130, 32: 130, 36: 129, 167: 128, 39: 128, 163: 127, 41: 127, 40: 125, 38: 125, 164: 124, 158: 123, 162: 123, 46: 120, 153: 120, 173: 118, 168: 117, 161: 117, 37: 116, 160: 116, 170: 113, 51: 113, 169: 112, 42: 111, 45: 109, 47: 109, 177: 109, 172: 108, 50: 107, 175: 106, 52: 104, 44: 104, 171: 103, 49: 102, 48: 100}

In [45]:
mant_dict[6060] = 119375

In [46]:
import pickle
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_dict_for_huff_castanets.pkl', 'wb')
pickle.dump(mant_dict,file)
file.close()

# Mantissa Dictionary for all the different possible sound files

### Glockenspiel

In [47]:
mant_dict_g = {0: 6416, 32768: 2268, 32769: 310, 2: 260, 3: 250, 4: 245, 32770: 245, 1: 242, 32772: 238, 32773: 228, 32775: 224, 32771: 219, 5: 218, 8: 216, 32776: 216, 32789: 212, 6: 211, 15: 211, 32786: 208, 11: 207, 32777: 207, 13: 206, 7: 205, 10: 203, 9: 201, 32778: 200, 32779: 200, 32788: 199, 14: 199, 32782: 198, 17: 198, 23: 196, 32783: 196, 12: 194, 22: 194, 32787: 192, 26: 192, 32784: 192, 24: 191, 32780: 191, 32781: 191, 32774: 190, 32792: 188, 18: 182, 32799: 182, 32785: 178, 29: 177, 20: 177, 32791: 176, 28: 176, 32794: 176, 19: 174, 32800: 174, 16: 173, 32803: 173, 32795: 173, 21: 171, 32796: 171, 32798: 171, 32793: 170, 27: 170, 25: 165, 47: 164, 33: 163, 44: 162, 32806: 162, 32797: 162, 42: 159, 69: 158, 32817: 157, 32828: 157, 36: 157, 32815: 155, 61: 155, 43: 154, 39: 153, 32790: 153, 32808: 150, 32: 150, 40: 150, 32801: 149, 32811: 149, 32804: 148, 31: 148, 32805: 147, 32812: 147, 64: 146, 34: 146, 37: 145, 41: 145, 35: 145, 32827: 144, 32807: 144, 32813: 143, 32816: 143, 32835: 143, 32826: 143, 32823: 143, 76: 143, 53: 142, 77: 142, 30: 142, 45: 142, 63: 142, 32802: 140, 52: 140, 32830: 140, 32860: 139, 32843: 139, 46: 138, 32822: 138, 32814: 137, 66: 137, 59: 137, 32839: 137, 32810: 137, 32824: 136, 51: 136, 116: 134, 49: 134, 32829: 134, 32842: 134, 32844: 134, 32818: 134, 88: 133, 32819: 133, 32833: 133, 73: 133, 32848: 132, 32820: 132, 38: 132, 32809: 132, 32851: 131, 32831: 131, 32840: 131, 32832: 130, 65: 130, 84: 130, 78: 129, 87: 129, 62: 129, 70: 129, 108: 129, 32853: 128, 32867: 128, 110: 128, 32886: 128, 85: 128, 32881: 127, 32836: 127, 32862: 127, 50: 126, 58: 126, 32850: 126, 56: 126, 32871: 125, 109: 124, 93: 123, 32837: 123, 82: 123, 74: 123, 54: 123, 32834: 122, 68: 122, 32900: 122, 32915: 122, 32873: 121, 32865: 121, 86: 120, 101: 120, 79: 120, 32870: 120, 72: 119, 32854: 119, 55: 119, 32821: 119, 32825: 119, 81: 119, 32919: 118, 75: 118, 91: 118, 57: 118, 111: 118, 60: 117, 32861: 117, 80: 116, 67: 116, 32889: 116, 132: 116, 103: 116, 89: 116, 32875: 115, 99: 115, 32883: 115, 32838: 115, 32856: 115, 32864: 115, 112: 114, 71: 114, 32847: 114, 32901: 114, 94: 114, 147: 114, 83: 113, 113: 113, 96: 113, 164: 113, 97: 112, 32903: 112, 32852: 111, 32876: 111, 95: 111, 32914: 111, 105: 111, 149: 111, 104: 110, 48: 110, 32926: 110, 152: 110, 122: 110, 32845: 110, 32911: 110, 139: 109, 32841: 109, 32895: 109, 32918: 109, 98: 109, 125: 109, 100: 109, 32866: 108, 107: 108, 90: 108, 32916: 108, 32894: 108, 32859: 108, 32896: 107, 127: 107, 32868: 107, 32902: 107, 32907: 107, 117: 107, 32924: 107, 119: 106, 141: 106, 32884: 106, 32933: 106, 32912: 105, 126: 105, 32857: 105, 92: 105, 106: 104, 32908: 104, 32874: 104, 32869: 104, 32904: 104, 124: 104, 32863: 104, 129: 104, 135: 104, 115: 103, 32947: 103, 32878: 103, 218: 103, 157: 103, 170: 103, 32913: 103, 32882: 103, 32934: 102, 140: 102, 168: 102, 32855: 102, 32941: 102, 146: 101, 134: 101, 32929: 101, 32880: 101, 172: 101, 114: 101, 32923: 101, 128: 100, 186: 100, 102: 100, 137: 100, 32921: 100, 32922: 100, 174: 100, 32872: 100}

In [48]:
mant_dict_g[6060] = 5191

In [49]:
import pickle
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_dict_for_huff_glockenspiel.pkl', 'wb')
pickle.dump(mant_dict_g,file)
file.close()

## Harpsichord

In [50]:
mant_dict_h = {0: 206713, 8: 67981, 4: 50138, 16: 45672, 2: 27011, 1: 25427, 32: 14141, 9: 7454, 17: 6932, 3: 5885, 5: 5617, 33: 5397, 10: 3779, 64: 2901, 6: 2699, 18: 2495, 11: 2393, 65: 2033, 12: 2031, 7: 1583, 19: 1490, 34: 1483, 13: 1381, 66: 1125, 20: 1083, 14: 976, 21: 819, 15: 774, 24: 762, 128: 719, 129: 695, 25: 648, 22: 633, 67: 624, 35: 590, 130: 555, 26: 552, 23: 487, 27: 463, 131: 434, 28: 421, 29: 364, 68: 354, 36: 349, 132: 347, 30: 336, 31: 283, 133: 260, 37: 215, 69: 203, 134: 195, 38: 183, 70: 183, 135: 177, 39: 148, 71: 139, 40: 136, 41: 130, 136: 129, 42: 117, 48: 106, 72: 101}

In [51]:
mant_dict_h[6060] = 5026

In [52]:
import pickle
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_dict_for_huff_harpsichord.pkl', 'wb')
pickle.dump(mant_dict_h,file)
file.close()

## Oboe

In [53]:
mant_dict_o =  {0: 219253, 16: 61770, 8: 55457, 32: 34001, 2: 33518, 4: 27270, 1: 22629, 17: 8886, 33: 6764, 3: 4265, 9: 4192, 64: 3726, 5: 2847, 18: 2550, 34: 2493, 6: 1645, 65: 1455, 10: 1392, 19: 1385, 35: 1332, 7: 1194, 20: 887, 11: 871, 36: 858, 12: 834, 66: 780, 37: 678, 21: 666, 13: 637, 24: 579, 67: 544, 22: 521, 25: 491, 14: 485, 38: 482, 23: 458, 15: 417, 39: 409, 26: 409, 128: 378, 40: 355, 68: 344, 28: 322, 27: 314, 29: 291, 48: 287, 41: 284, 129: 268, 42: 264, 30: 260, 69: 253, 50: 246, 49: 220, 31: 217, 52: 207, 51: 207, 43: 193, 53: 189, 130: 185, 45: 174, 54: 173, 70: 172, 71: 169, 44: 166, 46: 154, 56: 150, 55: 147, 59: 140, 58: 135, 60: 127, 131: 124, 57: 118, 61: 116, 63: 115, 47: 104, 132: 102, 73: 100}

In [54]:
mant_dict_o[6060] = 2584

In [55]:
import pickle
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_dict_for_huff_oboe.pkl', 'wb')
pickle.dump(mant_dict_o,file)
file.close()

## SPGM

In [56]:
mant_dict_s =  {0: 145197, 1: 51856, 8: 47921, 2: 42571, 4: 39513, 16: 26194, 9: 21637, 17: 16131, 3: 14835, 5: 12586, 10: 10771, 32: 9389, 18: 8996, 33: 8031, 19: 6320, 11: 5911, 6: 5645, 20: 4506, 34: 3755, 7: 3531, 21: 3510, 12: 3491, 22: 2756, 35: 2349, 13: 2189, 23: 2159, 24: 1861, 36: 1758, 25: 1488, 14: 1437, 37: 1385, 65: 1376, 38: 1127, 26: 1122, 66: 1089, 39: 1065, 15: 1029, 64: 929, 27: 874, 40: 853, 41: 746, 42: 717, 67: 717, 28: 671, 29: 633, 43: 581, 68: 561, 44: 556, 48: 539, 30: 527, 49: 486, 69: 459, 45: 450, 50: 430, 46: 414, 31: 391, 71: 364, 70: 362, 47: 355, 51: 353, 52: 324, 72: 295, 54: 292, 53: 284, 55: 247, 73: 246, 74: 214, 57: 205, 76: 193, 77: 183, 56: 183, 60: 180, 58: 179, 75: 175, 78: 167, 59: 165, 79: 160, 61: 147, 81: 137, 80: 125, 62: 120, 82: 118, 63: 109, 87: 107, 84: 102, 83: 100}

In [57]:
mant_dict_s[6060] = 2806

In [58]:
import pickle
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_dict_for_huff_spgm.pkl', 'wb')
pickle.dump(mant_dict_s,file)
file.close()

# Checking Mantissa Castanets Dictionary

In [33]:
file = open(r'C:\Users\audre\OneDrive\Documents\GitHub\Arithmetic_Audio_Coding\mantissa_dict_for_huff_all.pkl', 'rb')
mant_dict_all = pickle.load(file)
file.close()

In [38]:
vals = mant_dict_all.values()

In [39]:
sum_val = 0
for v in vals:
    if v < 100:
        sum_val += v
sum_val

119375